In [ ]:
library(here)


# Wastewater SARS-CoV-2 Data

After sequencing, we are left with the following data:

| Read | Sequence |
|------|----------|
| 1    | ATCA     |
| 2    | TCAC     |
| 3    | CACTA    |
| 4    | TCAATA   |
| 5    | TCTC     |
| 6    | TCTC     |

Alignment is the process of matching these reads up to a reference, which in this toy example results in the following:

| Read      | 1 | 2 | 3 | 4 | 5 | 6 | 7 | ... |
|-----------|---|---|---|---|---|---|---|-----|
| Reference | A | T | C | A | C | T | A | ... |
| 1         | A | T | C | A | - | - | - | ... |
| 2         | - | T | C | A | C | - | - | ... |
| 3         | - | - | C | A | C | T | A | ... |
| 4         | - | T | C | A | A | T | A | ... |
| 5         | - | T | C | T | C | - | - | ... |
| 6         | - | T | C | T | C | - | - | ... |
| Depth     | 1 | 5 | 6 | 6 | 5 | 2 | 2 | ... | 

A couple interesting things:

- This example is 1-indexed. Always good to know when you're working with genome positions.
- The read depth is the total number of reads that aligned with a given position on the genome.
- Read 4 has a mutation at position 5, where C changed to A.
    - This mutation would be recorded by gromstole as `C5A`.
    - There were 5 reads that were aligned to this position, so we have 1 mutation out of 5 reads.
- Similarly, mutation `A4T` has a count of 2 and a coverage of 6, leading to a frequency of 0.33.
- Paired-end reads are a little trickier to deal with and are different depending on who you ask.

In the [`gromstole`](https://github.com/PoonLab/gromstole) pipeline, these Single Nucleotide Variations will not include the reference base and the two mutations shown are simply `~5A` and `~4T`.

- Insertions and deletions can also occur - a deletion of 3 bases at position 5 would be labelled `-5.3` and an insertion of 3 bases at position 5 would be `+5.3`.

# Amino acid changes

With SARS-CoV-2, we know the positions of the amino acids (special groups of 3 bases), and the genome is split up into coding regions (e.g. the "spike protein").


In [2]:
provoc:::parse_mutations(c("~6037A", "~6037G"))


[1] "aa:orf1a:S1924R" "aa:orf1a:S1924R"

The output indicates that a mutation to A (from C) at location 6037 is an amino acid change from S to R at amino acid number 1924 in Open Reading Frame 1A. If the C had mutated to a G instead, we would get the same amino acid change.

Also note that the amino acid position 1924 is a little less than $6037/3$. The position of the AA is based on the start of the reading frame, which is a few hundred base pairs from the start. In the example below, notice that the point mutation at position 24000 shows up at AA position 813 in the spike protein.

In [3]:
provoc:::parse_mutations("~24000C")


[1] "aa:S:S813T"

# The Data

With all this in place, we can finally see the data used for modelling. From the counts and read depths (also called coverage), we can get a table that looks like this:

| position | label   | mutation | frequency | coverage |
|----------|---------|----------|-----------|----------|
| 10621    | ~10622T | aa:orf1a:T3453S | 0.043478260869565216 | 23 |
| 10665    | ~10666A | T10666A | 0.043478260869565216 | 23 |
| 10597    | ~10598G | aa:orf1a:Y3445D | 0.043478260869565216 | 23 |
| 10638    | ~10639G | T10639G | 0.043478260869565216 | 23 | 
| 10510    | ~10511T | aa:orf1a:D3416Y | 0.043478260869565216 | 23 |
| 11531    | -11532.1| del:11532:1 | 0.004291845493562232 | 233 |



The above file is specific to the `GromStole` pipeline. There are a few things to note:

- The first five have the same `coverage`. This is normal - the "ARTIC" protocol is a set of "primers" that try and gather data from specific positions (there's always the possibility for variability in this, so it's not perfect).
    - The first five have the same frequency as well, namely 1/23. This tells us that there was a count of 1, although this number is not reported.
- `Position` is 0-indexed.
- There's a "`label`" and "`mutation`" column.
    - The "`label`" uses a "~" to denote a point mutation, a "-" to denote a deletion, and a "+" to denote an insertion. The next part is the 1-indexed position. The final part is the new nucleotide, the number of nucleotides deleted, or the nucleotides added, respectively.
    - The `mutation` column tries to convert the label to an amino acid change. It starts with "`aa`" if it's a mutation, `del` if it's a deletion, and `ins` if it's an insertion. Most of the bioinformatics field uses aa mutations because they have information about the structure and function of the genome. However, there's not exactly a standardized notation for this. 

In addition to the file above (the so-called "mapped" file), there is a "covereage" file. This file simply reports the coverage at every position of the genome. This is very useful when searching for mutations that we know belong to lineages: if the mutation was not observed, we know that it's count was 0 but the mapped file will have no information about the coverage. A count of 0 with a coverage of 1 doesn't mean much, but a count of 0 when the coverage is 5000 means the mutation (and therefore the lineage) is probably not present either.

# Data Post-Processing

There are a few main approaches to processing the data, depending on the purpose of the study. Some studies will be using a set of mutations that are known to be present in various lineages, other studies need observations of each mutation on each day.

## Mutation List-based Analyses

This is pretty straightforward: if we have a set of mutations that we are looking for, we simply find those in the data. Right? Well, we also care about whether those mutations were *not* present. As noted before, 0/1 and 0/5000 are different amounts of information from missing mutations. The processing steps are thus:

1. Find the mutations of interest that are present in the data.
2. For mutations that are not present, use the coverage file to find out exactly how "not present" they are.
3. For convenience, include information about which lineages each mutation appears in.

The above process is done for all of the samples that we have. The resulting file looks like this:



## 


In [4]:
jahn <- read.csv(here("data/jahn_variants.csv"))
head(jahn[, c("mutation", "sample", "date", "count", "coverage", "var_B.1.1.7", "var_B.1.351", "var_B.1.617.2", "var_P.1")])


,mutation,sample,date,count,coverage,var_B.1.1.7,var_B.1.351,var_B.1.617.2,var_P.1
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>
1,aa:E:P71L,ERR5922198,2021-02-13,1,3018,0,1,0,0
2,aa:E:P71L,ERR5922199,2021-02-12,1,1177,0,1,0,0
3,aa:E:P71L,ERR5922200,2021-02-10,2,4726,0,1,0,0
4,aa:E:P71L,ERR5922201,2021-02-09,2,4032,0,1,0,0
5,aa:E:P71L,ERR5922202,2021-02-07,1,4953,0,1,0,0
6,aa:E:P71L,ERR5922203,2021-02-06,0,11,0,1,0,0


As you can see, the mutations are still in GromStole's aa format. 

- Sample names tell us which wastewater sample the data come from.
- In this example, the data are sorted by mutation, which is why we see the same mutation on several different dates 
    - They are also present in several different locations, which is not shown here.
- The "`var_*`" columns tell us that mutation `aa:E:P71L` is present in B.1.351 (Beta lineage), but not in the other lineages. 
    - During the study period Beta was pretty low, hence the low counts relative to the coverage.

## Mutation-centric analysis, especially clustering

There are several analyses which are not based on knowing which mutations to look for. This requires a slightly different data processing pipeline. We must strike a balance between information and file size. This is achieved by:

1. Find all mutations that had a frequency above 20% at least once in the study period.
    - Justification: If a mutation is never present in more than 20% of the sample, then it probably doesn't have much information.
2. Find those mutations in all samples (using coverage files to make up for mutations with 0 count in a given sample).

I have this as the second version of processing the data, but this is actually the default. The lineage-based processing can be accomplished by further processing these files. 

The resultant file is the same as the one above, but without the "`var_*`" columns. The difference is the amount of data.